In [12]:
import warnings
warnings.filterwarnings('ignore')

try:
    %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

In [13]:
# path 지정 
train_dir='./lions_and_tigers/train'
train_lions_dir=os.path.join(train_dir,'lion')
train_tigers_dir=os.path.join(train_dir,'tiger')

test_dir='./lions_and_tigers/test'
test_lions_dir=os.path.join(test_dir,'lion')
test_tigers_dir=os.path.join(test_dir,'tiger')

val_dir='./lions_and_tigers/test'
val_lions_dir=os.path.join(val_dir,'lion')
val_tigers_dir=os.path.join(val_dir,'tiger')

In [14]:
# 데이터 전처리 (증대x, 원본 데이터에 약간 변화를 줘서 다양성을 늘림)
train_datagen=ImageDataGenerator(rescale=1./255, # 값을 0~1사이로 바꿈(min-max scaler)
                                shear_range=0.2, # 찌그러트림
                                zoom_range=0.2,
                                horizontal_flip=True) # 수평 방향으로 flip하는가 
# 테스트 데이터에 변화를 줄 필요 없음 (리스케일만 함)
test_datagen=ImageDataGenerator(rescale=1./255)

# flow_from_directory: 디렉토리로부터 파일 가져오기 
train_generator=train_datagen.flow_from_directory(train_dir,
                                                target_size=(150,150),
                                                batch_size=20, # 20개씩 가져옴 
                                                class_mode='binary')

validation_generator=train_datagen.flow_from_directory(val_dir,
                                                target_size=(150,150),
                                                batch_size=20,
                                                class_mode='binary')

test_generator=test_datagen.flow_from_directory(test_dir,
                                                target_size=(150,150),
                                                batch_size=20,
                                                class_mode='binary')

Found 360 images belonging to 2 classes.
Found 40 images belonging to 2 classes.
Found 40 images belonging to 2 classes.


In [15]:
for data_batch, labels_batch in train_generator:
    print('배치 데이터 크기:', data_batch.shape)
    print('배치 레이블 크기:',labels_batch.shape)
    break

배치 데이터 크기: (20, 150, 150, 3)
배치 레이블 크기: (20,)


In [16]:
model=tf.keras.Sequential([
    tf.keras.layers.Conv2D(input_shape=(150,150,3),kernel_size=(3,3),filters=32),
    tf.keras.layers.MaxPool2D(strides=(2,2)),
    tf.keras.layers.Conv2D(kernel_size=(3,3),filters=64),
    tf.keras.layers.MaxPool2D(strides=(2,2)),
    tf.keras.layers.Conv2D(kernel_size=(3,3),filters=128),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=128,activation='relu'),
    tf.keras.layers.Dropout(rate=0.3),
    tf.keras.layers.Dense(units=1,activation='sigmoid')
])

model.compile(optimizer=tf.keras.optimizers.Adam(), loss='binary_crossentropy',
             metrics=['accuracy'])
model.summary()
# 1 epoch당 18번(train_size:360//batch_size:20)번 step, 2step(val_size:40//batch_size:20)당 1번 검증 -> 1 epoch당 18번 step, 9번 검증 
history=model.fit(train_generator,steps_per_epoch=360//20, epochs=5,
                 validation_data=validation_generator,validation_steps=40//20) 

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
flatten_2 (Flatten)          (None, 147968)            0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)              

In [17]:
model.evaluate(test_generator,verbose=0)

[0.6931479573249817, 0.5]